In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import rowcol
from scipy.ndimage import distance_transform_edt
from tqdm import tqdm

Resample tif to be 3m perhaps?

In [3]:
# === Paths ===
input_ndvi_tif = "D:/terrain_generation_project/NAIP_processed/naip_ndvi_1_8.tif"
output_gpkg = "D:/terrain_generation_project/NAIP_processed/vegetation_polygons.gpkg"
layer_name = "vegetation"

In [ ]:
# === Parameters ===
input_ndvi_tif = "D:/terrain_generation_project/NAIP_processed/naip_ndvi_1_8.tif"
structure_gpkg = "D:/terrain_generation_project/structures_with_distances.gpkg"
structure_layer = "postfire"
output_gpkg = "D:/terrain_generation_project/NAIP_processed/structures_with_distance_ndvi.gpkg"
output_layer = "structures_with_distance_ndvi"
ndvi_min = 0.3
ndvi_max = 0.8

# === Load NDVI and create vegetation mask ===
print("Loading NDVI raster...")
with rasterio.open(input_ndvi_tif) as src:
    ndvi = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

    # Make sure to ignore nodata
    valid_mask = (ndvi != nodata) if nodata is not None else np.ones_like(ndvi, dtype=bool)

    # Create binary vegetation mask
    veg_mask = (ndvi >= ndvi_min) & (ndvi <= ndvi_max) & valid_mask

    # Invert for distance transform (True = non-veg)
    inverse_mask = ~veg_mask

    print("Computing distance transform...")
    distance_pixels = distance_transform_edt(inverse_mask)

    # Convert to meters using pixel size (assumes square pixels)
    pixel_size = transform.a
    distance_meters = distance_pixels * pixel_size

# === Load structure points ===
print("Loading structure points...")
structures = gpd.read_file(structure_gpkg, layer=structure_layer).to_crs(crs)

# === Get distance to nearest vegetation pixel for each point ===
print("Calculating distances to vegetation...")
distances = []
for point in tqdm(structures.geometry, desc="Processing points"):
    try:
        row, col = rowcol(transform, point.x, point.y)
        dist = distance_meters[row, col]
    except IndexError:
        dist = np.nan  # Outside raster
    distances.append(dist)

structures["ndvi_dist_m"] = distances

# === Save result ===
print(f"Saving output to {output_gpkg}...")
structures.to_file(output_gpkg, layer=output_layer, driver="GPKG")

print("Done.")


Extracting vegetation polygons:   0%|          | 0/143631423 [00:00<?, ?it/s]